In [2]:
from Classes.LoadModel import BaseModel
from Classes.Predict import Prediction
from Classes.Summarize import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.layers import Dense, Flatten, Layer, Input, Dropout, Activation, Convolution2D, MaxPooling2D

from BaseModels import Facenet, VGGFace
from AttModels import Age, Emotion, Gender, Race
import tensorflow as tf

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [3]:
def load_images(file_path, att='All'):
    list_image = os.listdir(file_path)
    final_list = []
    for i in list_image[:5000]:
        if att.lower() =='emotion':
            img = tf.keras.preprocessing.image.load_img(os.path.join(file_path, i), target_size=(48,48), color_mode="grayscale")
        else:
            img = tf.keras.preprocessing.image.load_img(os.path.join(file_path, i), target_size=(224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = np.expand_dims(img_array, 0)
        final_list.append(img_array)
    return final_list

In [4]:
def top_layers_vggface1(model_base):
    model_base.trainable = False
    model = Sequential()
    model.add(model_base)
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [5]:
def top_layers_vggface7(model_base):
    model_base.trainable = False
    model = Sequential()
    model.add(model_base)
    model.add(Flatten())
    model.add(Dense(256,activation='relu'))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(5,activation='softmax'))
    return model

# With NN

In [6]:
best_model_path = '/home/ubuntu/sheryl/Facial_Attributes_Detection/model-att'
image_path = '/home/ubuntu/sheryl/face_att_sheryl'
attribute_list = {'Wearing_Hat':'vggface', 'Eyeglasses':'vggface', 'Wearing_Necktie':'vggface', 'Hair_color':'vggface', 'Wearing_Necklace':'vggface', 'Mustache':'vggface', 'No_Beard':'vggface', 'Wearing_Earrings':'vggface', 'Age': 'age', 'Gender':'gender', 'Race':'race','Emotion':'emotion'}

In [7]:
from time import time

def return_prediction_time(att, attribute_list, image_list, batch=32):
    model_name = attribute_list[att]
    basemodel = BaseModel(model_name)
    model = basemodel.load_model(False)
    print(f'\nModel {model_name} for {att} Loaded!')
    if att.lower() not in ['race', 'gender', 'emotion', 'age', 'hair_color'] :
        model = top_layers_vggface1(model)
        print('vggface1 layers loaded')
    elif 'hair' in att.lower():
        model = top_layers_vggface7(model)
        print('vggface7 layers loaded')
    else:
        model = model 
        print(att, 'model loaded')
    print(f'\nTop layers for {att} added !')

    weight_file_name = ''     
    if att.lower() in ['race', 'gender', 'age', 'emotion']:
        weight_file_name = att.lower() + '_model_weights.h5'
    elif 'hair' in att.lower():
        weight_file_name = 'vggface7_Hair_color.h5'
    else:
        weight_file_name = 'vggface1_' + att + '.h5'
    
    print('Loading best weights...')
    model.load_weights(os.path.join(best_model_path, weight_file_name))
    
    print('Compile Model...')
    if 'hair' not in att.lower():
        model.compile(RMSprop(lr=0.0001, decay=1e-6), loss='binary_crossentropy', metrics=["accuracy"])
    else:
        model.compile(RMSprop(lr=0.0001, decay=1e-6), loss='categorical_crossentropy', metrics=["accuracy"])       
    
    print('Starts Prediction...')
    
    time_list = []
    batch_list =[]
    if att == 'Emotion':
        start_time = time()
        for i in image_list:
            predictions = model.predict(i)
        total_time = round((time() - start_time), 2)
        time_per_image = total_time / 5000
        time_per_image = round(time_per_image, 4)                       
        print('The prediction time for', att, 'with', model_name, 'model is', total_time, 'seconds')
        print(att, 'prediction time per image is', time_per_image, 'seconds')
    else: 
        start_time = time()
        predictions = model.predict(image_list, batch_size=batch)
        total_time = round((time() - start_time), 2)

        time_per_image = total_time / 5000
        time_per_image = round(time_per_image, 4)                       
        print('The prediction time for', att, 'with', model_name, 'model is', total_time, 'seconds', 'with batch', batch)
        print(att, 'prediction time per image is', time_per_image, 'seconds', 'with batch', batch)
        print('\n')
    return total_time, time_per_image

In [8]:
# batch_10 =[]
# batch_32 =[]
# batch_100 = []
# batch_1000 =[]
# for att in list(attribute_list.keys()):
#     image_list = load_images(image_path,'emotion')
#     for batch in [10, 32, 100, 1000]:
#         total_time, time_per_image = return_prediction_time(att, attribute_list, image_list, batch)
#         if batch == 10:
#             batch_10.append(time_per_image)
#         elif batch == 100:
#             batch_100.append(time_per_image)
#         elif batch == 32:
#             batch_32.append(time_per_image)
#         else:
#             batch_1000.append(time_per_image)

In [9]:
attribute_list = {'Wearing_Hat':'vggface', 'Eyeglasses':'vggface', 'Wearing_Necktie':'vggface', 'Hair_color':'vggface', 'Wearing_Necklace':'vggface', 'Mustache':'vggface', 'No_Beard':'vggface', 'Wearing_Earrings':'vggface', 'Age': 'age', 'Gender':'gender', 'Race':'race','Emotion':'emotion'}

batch_10 =[0.005,
 0.0047,
 0.005,
 0.005,
 0.0048,
 0.005,
 0.0047,
 0.0049,
 0.0052,
 0.0048,
 0.0049, 
0.0358]
batch_32 =[0.0047,
 0.0046,
 0.005,
 0.0048,
 0.0047,
 0.005,
 0.0047,
 0.0048,
 0.005,
 0.0047,
 0.0049, 
0.0358]
batch_100 = [0.0048,
 0.0047,
 0.0051,
 0.0048,
 0.0048,
 0.005,
 0.0047,
 0.0048,
 0.0047,
 0.0047,
 0.0052, 
0.0358]
batch_1000 =[0.0045,
 0.0047,
 0.0047,
 0.0046,
 0.0047,
 0.0047,
 0.0048,
 0.0049,
 0.0046,
 0.0048,
 0.0048,
0.0358]
attribute = list(attribute_list.keys())

In [10]:
import pandas as pd
my_dict = {'Attribute':attribute,'Time_per_image(sec) \n Batch10':batch_10,'Time_per_image(sec) \n Batch32':batch_32, 'Time_per_image(sec) \n Batch100':batch_100, 'Time_per_image(sec) \n Batch1000':batch_1000 }
df = pd.DataFrame(my_dict).set_index('Attribute')
df

,Time_per_image(sec) Batch10,Time_per_image(sec) Batch32,Time_per_image(sec) Batch100,Time_per_image(sec) Batch1000
Attribute,,,,
Wearing_Hat,0.0050,0.0047,0.0048,0.0045
Eyeglasses,0.0047,0.0046,0.0047,0.0047
Wearing_Necktie,0.0050,0.0050,0.0051,0.0047
Hair_color,0.0050,0.0048,0.0048,0.0046
Wearing_Necklace,0.0048,0.0047,0.0048,0.0047
Mustache,0.0050,0.0050,0.0050,0.0047
No_Beard,0.0047,0.0047,0.0047,0.0048
Wearing_Earrings,0.0049,0.0048,0.0048,0.0049
Age,0.0052,0.0050,0.0047,0.0046


# With Embedding

In [10]:
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import Model

def loading_embedding(imagepath, model):
    """
    :param imagepath: path to the image folder
    :param model: model
    :param data: dataset
    :param layer_num: position of the layer starting from the end of the summary. Determine where to cut the model
    This function takes a model, cut the layer and save the embedding for each images in a dataset
    """
    list_x = []
    list_image = os.listdir(imagepath)
    for image in list_image[:5000]:
        img = image.load_img(os.path.join(imagepath, img), target_size=(160, 160))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = x.astype('float32') / 255.
        list_x.append(x)
    feature_x = np.vstack(list_x)
    feature = model.predict(feature_x)
    
    return feature

In [ ]:
# For embeddig 

from time import time
from Classes.LoadModel import BaseModel
from Classes.Summarize import *
from Classes.BaseCls import *
from time import time

pkl_model_path = 
image_path = '/home/ubuntu/sheryl/face_att_sheryl'
attribute_list = ['Wearing_Hat', 'Eyeglasses', 'Wearing_Necktie', 'Wearing_Necklace', 'Mustache', 'No_Beard', 'Wearing_Earrings', 'Age', 'Gender', 'Race','Emotion']

model_name = 'facenet'
print(f'\nLoading {model_name} Base Model...')
basemodel = BaseModel(model_name)
model = basemodel.load_model(True)
model = Model(inputs=model.input, outputs=model.layers[-1].output)
time_list_embg = []
print('\nStarting image embedding')
time_embedding = time()
image_embedding  = loading_embedding(imagepath, model)
final_time_embedding = time() - time_embedding

print('Time for image embedding is', final_time_embedding)

for att in attribute_list:
    
    time_prediction = time()
    pkl_filename = att +  '.pkl'
    
    with open(pkl_filename, 'rb') as file:
        clf_model = pickle.load(file)
        
    print('Start prediction for', att)
    
    prediction = clf_model.predict(image_embedding)
    
    final_time_prediction = time() - time_prediction
    
    total_time = final_time_prediction + final_time_embedding
    time_list_embg.append(total_time)
    print('Prediction time for one image', att, 'is', final_time_prediction / 5000)
    print('Total time for one image', att, 'is', total_time / 5000)        